In [0]:
%run ./utilityFunctions

In [0]:
# Initialize Variables
import pandas as pd
capitalCityTable = spark.table("gold.capitalcitylist")
capitalCityTablePd = capitalCityTable.toPandas()
appIdStr = "<Redacted>"
excludeStr = "minutely,alerts"

In [0]:
# Store the data
from pyspark.sql.functions import col, lit
currentTime = getCurrentTimeUTC()
databaseName = "bronze"
for index, row in capitalCityTablePd.iterrows():
    continentName = row['ContinentName']
    countryName = row['CountryName']
    capitalName = row['CapitalName']
    lat = row['CapitalLatitude']
    lon = row['CapitalLongitude']
    df = getWeatherData(lat, lon, appIdStr, excludeStr)
    df = df.withColumn("continent", lit(continentName)).withColumn("country", lit(countryName)).withColumn("capital", lit(capitalName)).withColumn("currentTime", lit(currentTime))
    tableName = "rawdata" + capitalName.lower() + currentTime
    writeDfToDeltaTable(df, databaseName, tableName, "overwrite")
